In [84]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Functions
# Add the directory containing the module to sys.path
sys.path.append(os.path.abspath('functions'))
from preprocessing import read_csv_files, check_missing_values, forward_fill, backward_fill, linear_interpolation, calculate_volatility, adf_test, check_date_range, extract_date_range, map_date_range, create_volatility_df, stationary_transformation
from spillover import calculate_avg_spillover_table, calculate_net_pairwise_spillover_table

## Data Preprocessing

In [85]:
# Retrieve a list of DataFrames
dataframes = read_csv_files("data")

In [86]:
# Convert each column in every DataFrame to lowercase
for key in dataframes:
    dataframes[key].columns = map(str.lower, dataframes[key].columns)

Converting the `date` column to datetime object, this process has to be handled individually since each DataFrame has a different date format. We cannot let Pandas infers the date format for each DataFrame since it can be prone to infer the wrong format. Below are the format of each DataFrame:
- Philippines: MM/DD/YYYY
- Singapore: MM/DD/YYYY
- India: YYYY-MM-DD
- United Kingdom: DD/MM/YYYY
- Mexico: YYYY-MM-DD
- Japan: YYYY-MM-DD
- Vietnam: DD/MM/YYYY
- Korea: YYYY-MM-DD
- Thailand: YYYY-MM-DD
- Brazil: YYYY-MM-DD
- Malaysia: DD/MM/YYYY
- Switzerland: YYYY-MM-DD
- China: DD/MM/YYYY
- Russia: YYYY-MM-DD
- United States: YYYY-MM-DD

In [87]:
date_format_mapping = {
  'philippines': '%m/%d/%Y',
  'singapore': '%m/%d/%Y',
  'india': '%Y-%m-%d',
  'uk': '%d/%m/%Y',
  'mexico': '%Y-%m-%d',
  'japan': '%Y-%m-%d',
  'vietnam': '%d/%m/%Y',
  'korea': '%Y-%m-%d',
  'thailand': '%Y-%m-%d',
  'brazil': '%Y-%m-%d',
  'malaysia': '%d/%m/%Y',
  'switzerland': '%Y-%m-%d',
  'china': '%d/%m/%Y',
  'russia': '%Y-%m-%d',
  'us': '%Y-%m-%d',
}

# Convert the date columns to datetime objects
for key in dataframes:
    try:
      dataframes[key]['date'] = pd.to_datetime(
        dataframes[key]['date'], 
        format=date_format_mapping[key]
      )
    except Exception as e:
      print(f"Error occurred for country: {key}")
      print(f"Error message: {str(e)}")

In [88]:
# Sort the dataframes by date in ascending order
for key in dataframes:
    dataframes[key] = dataframes[key].sort_values(by='date')

In [89]:
# Reset the index of the dataframes
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

In [90]:
# Extract only open, high, low, close columns
for key in dataframes:
    dataframes[key] = dataframes[key][['date', 'open', 'high', 'low', 'close']]

In [91]:
for df in dataframes.values():
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5597 entries, 0 to 5596
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5597 non-null   datetime64[ns]
 1   open    5597 non-null   object        
 2   high    5597 non-null   object        
 3   low     5597 non-null   object        
 4   close   5597 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 218.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5656 entries, 0 to 5655
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5656 non-null   datetime64[ns]
 1   open    5607 non-null   float64       
 2   high    5607 non-null   float64       
 3   low     5607 non-null   float64       
 4   close   5607 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 221.1 KB
None
<class 'pandas.core.frame.DataFrame'>
Ra

Since the columns open, high, low, close have different dtypes (`float64` and `object`) for different DataFrame, they should be converted to `float64`.

In [92]:
# Convert open, high, low, close columns to float
for key in dataframes:
  for col in ['open', 'high', 'low', 'close']:
    if dataframes[key][col].dtype == 'object':
      dataframes[key][col] = dataframes[key][col].str.replace(',', '').astype(float)


In [93]:
# Check whether the columns have been converted to float
for df in dataframes.values():
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5597 entries, 0 to 5596
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5597 non-null   datetime64[ns]
 1   open    5597 non-null   float64       
 2   high    5597 non-null   float64       
 3   low     5597 non-null   float64       
 4   close   5597 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 218.8 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5656 entries, 0 to 5655
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5656 non-null   datetime64[ns]
 1   open    5607 non-null   float64       
 2   high    5607 non-null   float64       
 3   low     5607 non-null   float64       
 4   close   5607 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 221.1 KB
None
<class 'pandas.core.frame.DataFrame'>
Ra

##### Data imputation

This section aims to handle the missing values present in each DataFrame. Since these stock datasets will eventually be fed into a Vector Auto-regressive (VAR) model, it is quite important to choose a data imputation method that will preserve the temporal and cross-sectional relationships among countries.

Data imputation will be performed in two separate sections since one of our following step will create more missing values. A suitable method is chosen for each step, details are below:

1. Using linear interpolation to fill in the NA values originally presented in each dataset.
2. Dividing the imputed datasets from step 1 into 5 windows as defined in the Window Extraction step, resulting in 5 dictionaries containing DataFrames of countries with available data in each period.
3. The goal of this step is to normalize the date range in each time window since every country has their own public holidays, making the number of available daily trading data different for each country. Performing this step will ensure that holidays data will be included and the number of daily data available for each country within a time window is the same. For each DataFrame in a time window, map the available data into a new empty DataFrame with a date column containing every business days (excluding public holidays) of the associated period. Then, missing data from the mapped DataFrame will be imputed using forward filling method.
4. Calculate daily volatility for each DataFrame in every time window.
5. Create a single DataFrame for each time window with each column representing a country's volatility calculation.


In [94]:
check_missing_values(dataframes)

Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date      0
open     49
high     49
low      49
close    49
dtype: int64


Missing values for japan:
date      0
open     93
high     93
low      93
close    93
dtype: int64


Missing values for korea:
date      0
open     66
high     66
low      66
close    66
dtype: int64


Missing values for thailand:
date      0
open     73
high     73
low      73
close    73
dtype: int64


Missing values for switzerland:
date      0
open     52
high     52
low      52
close    52
dtype: int64


Missing values for us:
date     0
open     0
high     0
low      0
close    0
dtype: int64




In [95]:
# Impute missing values in the dataframes using linear interpolation method
dataframes = linear_interpolation(dataframes)

In [96]:
# Check whether the missing values have been imputed
check_missing_values(dataframes)

Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for switzerland:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for us:
date     0
open     0
high     0
low      0
close    0
dtype: int64




##### Window extraction

In this section, I will perform window extraction for the following time period:
- **Window 1:** 02.01.2002 to 17.09.2007
- **Window 2:** 18.09.2007 to 27.10.2011 (India's dataset starts from 18.09.2007)
- **Window 3:** 28.10.2011 to 31.12.2018 (Philippines's dataset starts from 28.10.2011)
- **Window 4:** 02.01.2019 to 31.12.2022 
- **Window 5:** 02.01.2023 to 30.04.2024 

##### 02.01.2002 - 17.09.2007: Recovery period from Dot-com Bubble

In [97]:
# Check whether each DataFrame contains data from 02.01.2002 to 17.09.2007
window_1_start_date = '2002-01-02'
window_1_end_date = '2007-09-17'
check_date_range(dataframes, window_1_start_date, window_1_end_date)

{'uk': True,
 'mexico': True,
 'japan': True,
 'korea': True,
 'thailand': True,
 'switzerland': True,
 'us': True}

The following countries do not have data available for this time period:
1. Philippine
2. Singapore
3. India
4. Vietnam
5. Brazil
6. Malaysia
7. Russia

These 7 countries will be be considered in VAR experiment for this time period, with the addition of China since it only has data from 2005.

In [98]:
dfs_window_1 = extract_date_range(dataframes, window_1_start_date, window_1_end_date)

# Check whether each DataFrame contains data from 02.01.2002 to 17.09.2007
dfs_window_1

{'uk':            date    open    high     low   close
 0    2002-01-02  5217.4  5261.9  5195.2  5218.3
 1    2002-01-03  5218.3  5328.4  5218.3  5318.8
 2    2002-01-04  5318.8  5362.3  5313.9  5323.8
 3    2002-01-07  5323.8  5354.2  5277.7  5293.6
 4    2002-01-08  5293.6  5307.3  5238.7  5250.4
 ...         ...     ...     ...     ...     ...
 1438 2007-09-11  6134.1  6280.7  6134.1  6280.7
 1439 2007-09-12  6280.7  6317.1  6232.0  6306.2
 1440 2007-09-13  6306.2  6374.5  6280.2  6363.9
 1441 2007-09-14  6363.9  6363.9  6209.1  6289.3
 1442 2007-09-17  6289.3  6289.3  6168.0  6182.8
 
 [1443 rows x 5 columns],
 'mexico':            date          open          high           low         close
 0    2002-01-02   6386.180176   6415.850098   6382.020020   6410.049805
 1    2002-01-03   6410.089844   6603.750000   6410.089844   6603.750000
 2    2002-01-04   6604.069824   6655.689941   6604.069824   6612.080078
 3    2002-01-07   6615.870117   6635.950195   6556.310059   6565.439941
 4 

In [99]:
# Remove countries that do not have data for the specified date range (02.01.2002 to 17.09.2007) from the dictionary
for key in list(dfs_window_1.keys()):
  if dfs_window_1[key].empty:
    del dfs_window_1[key]

# Remove the 'china' DataFrame from the dictionary
del dfs_window_1['china']

KeyError: 'china'

In [ ]:
len(dfs_window_1)

7

In [ ]:
# Map dfs_window_1 to a new date range
dfs_window_1 = map_date_range(dfs_window_1, window_1_start_date, window_1_end_date)

In [ ]:
# Check for missing values in the dfs_window_1
check_missing_values(dfs_window_1)

Missing values for uk:
date      0
open     46
high     46
low      46
close    46
dtype: int64


Missing values for mexico:
date     0
open     5
high     5
low      5
close    5
dtype: int64


Missing values for japan:
date      0
open     13
high     13
low      13
close    13
dtype: int64


Missing values for korea:
date      0
open     21
high     21
low      21
close    21
dtype: int64


Missing values for thailand:
date      0
open     27
high     27
low      27
close    27
dtype: int64


Missing values for switzerland:
date      0
open     10
high     10
low      10
close    10
dtype: int64


Missing values for us:
date      0
open     52
high     52
low      52
close    52
dtype: int64




In [ ]:
# Impute missing values in dfs_window_1 using forward fill method
dfs_window_1 = forward_fill(dfs_window_1)

/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/functions/preprocessing.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


In [ ]:
# Check for missing values in the dfs_window_1
check_missing_values(dfs_window_1)

Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     2
high     2
low      2
close    2
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for switzerland:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for us:
date     0
open     0
high     0
low      0
close    0
dtype: int64




Japan still has 2 missing values. These missing values are probably the first two days of the period. Let's use backward fill to handle these two missing values.

In [ ]:
dfs_window_1 = backward_fill(dfs_window_1)

/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/functions/preprocessing.py:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


In [ ]:
# Check for missing values in the dfs_window_1 again to make sure the missing values have been imputed
check_missing_values(dfs_window_1)

Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for switzerland:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for us:
date     0
open     0
high     0
low      0
close    0
dtype: int64




In [ ]:
# Calculate the volatility for each country
dfs_window_1 = calculate_volatility(dfs_window_1)

In [ ]:
# Create volatility DataFrame for window 1
df_window_1 = create_volatility_df(dfs_window_1)

In [ ]:
# Print out the first 5 rows of the volatility DataFrame
df_window_1.head()

,date,uk,mexico,japan,korea,thailand,switzerland,us
0,2002-01-02,17.955641,5.943942,19.791884,51.994083,12.228153,20.881106,21.664973
1,2002-01-03,18.456913,21.957613,19.791884,25.862243,17.854374,12.890345,8.171472
2,2002-01-04,12.755441,10.922138,19.791884,21.446678,16.412695,11.019224,13.936110
3,2002-01-07,19.064627,14.299720,16.900705,41.700360,18.717009,17.311078,14.120694
4,2002-01-08,15.443547,10.420974,17.203246,22.994618,12.864248,12.207484,11.487229


Performing ADF Stationarity Test

In [ ]:
for country in df_window_1.drop(columns=['date']).columns:
    adf_test(df_window_1, country)

Results of Augmented Dickey-Fuller Test for uk:
ADF Statistic: -3.312306568085512
p-value: 0.014337253065932897
Critical Values:
	1%: -3.434803261722705
	5%: -2.863506823125904
	10%: -2.567817117325164
Reject the null hypothesis. The time series uk is stationary.


Results of Augmented Dickey-Fuller Test for mexico:
ADF Statistic: -4.9554957820537355
p-value: 2.7162992260244777e-05
Critical Values:
	1%: -3.4347941822385923
	5%: -2.863502816068378
	10%: -2.5678149835134185
Reject the null hypothesis. The time series mexico is stationary.


Results of Augmented Dickey-Fuller Test for japan:
ADF Statistic: -2.976810034469136
p-value: 0.03711753950882737
Critical Values:
	1%: -3.4348184769200056
	5%: -2.86351353803652
	10%: -2.567820693109295
Reject the null hypothesis. The time series japan is stationary.


Results of Augmented Dickey-Fuller Test for korea:
ADF Statistic: -3.968076683867417
p-value: 0.001587211913799357
Critical Values:
	1%: -3.434806296467238
	5%: -2.8635081624499494
	10

In [ ]:
df_window_1 = stationary_transformation(df_window_1, 'switzerland')

In [ ]:
df_window_1 = df_window_1.dropna()

In [ ]:
df_window_1.head()

,date,uk,mexico,japan,korea,thailand,switzerland,us
1,2002-01-03,18.456913,21.957613,19.791884,25.862243,17.854374,-7.990762,8.171472
2,2002-01-04,12.755441,10.922138,19.791884,21.446678,16.412695,-1.871120,13.936110
3,2002-01-07,19.064627,14.299720,16.900705,41.700360,18.717009,6.291853,14.120694
4,2002-01-08,15.443547,10.420974,17.203246,22.994618,12.864248,-5.103593,11.487229
5,2002-01-09,12.444015,19.866337,14.393731,28.754780,19.555289,1.383995,26.480295


In [ ]:
# Peform ADF test again to make sure that the data is stationary
for country in df_window_1.drop(columns=['date']).columns:
    adf_test(df_window_1, country)

Results of Augmented Dickey-Fuller Test for uk:
ADF Statistic: -3.3073170991136145
p-value: 0.014554551410809547
Critical Values:
	1%: -3.434806296467238
	5%: -2.8635081624499494
	10%: -2.567817830533574
Reject the null hypothesis. The time series uk is stationary.


Results of Augmented Dickey-Fuller Test for mexico:
ADF Statistic: -4.944605249833368
p-value: 2.8533491841105752e-05
Critical Values:
	1%: -3.4347972046167983
	5%: -2.863504149938827
	10%: -2.567815693817062
Reject the null hypothesis. The time series mexico is stationary.


Results of Augmented Dickey-Fuller Test for japan:
ADF Statistic: -2.958356371036136
p-value: 0.038970674114539444
Critical Values:
	1%: -3.434821532444398
	5%: -2.863514886524506
	10%: -2.5678214111987328
Reject the null hypothesis. The time series japan is stationary.


Results of Augmented Dickey-Fuller Test for korea:
ADF Statistic: -3.9549371953703267
p-value: 0.0016657517226612143
Critical Values:
	1%: -3.4348093353507494
	5%: -2.863509503599295

##### 18.09.2007 - 27.10.2011: Global Financial Crisis

In [ ]:
# Check whether each DataFrame contains data from 18.09.2007 to 27.10.2011
window_2_start_date = '2007-09-18'
window_2_end_date = '2011-10-27'
check_date_range(dataframes, window_2_start_date, window_2_end_date)

{'philippines': False,
 'singapore': True,
 'india': True,
 'uk': True,
 'mexico': True,
 'japan': True,
 'vietnam': True,
 'korea': True,
 'thailand': True,
 'brazil': True,
 'malaysia': True,
 'switzerland': True,
 'china': True,
 'russia': False,
 'us': True}

Most countries have data from 18.09.2007 to 27.10.2011, with the exception of Philippines and Russia. In addition, there are Singapore, Vietnam, Brazil, and Malaysia which only has data from 07.03.2011, 05.01.2009, 24.02.2010, and 20.05.2010 respectively. These 4 countries will be remove from this time window.

In [ ]:
dfs_window_2 = extract_date_range(dataframes, window_2_start_date, window_2_end_date)

# Check whether each DataFrame contains data from 18.09.2007 to 27.10.2011
dfs_window_2

{'philippines': Empty DataFrame
 Columns: [date, open, high, low, close]
 Index: [],
 'singapore':           date     open     high      low    close
 0   2011-03-07  3054.43  3074.93  3047.59  3066.52
 1   2011-03-08  3063.58  3105.90  3062.52  3103.84
 2   2011-03-09  3110.54  3113.02  3087.84  3092.90
 3   2011-03-10  3093.79  3094.38  3070.55  3075.44
 4   2011-03-11  3053.07  3058.27  3024.17  3043.49
 ..         ...      ...      ...      ...      ...
 158 2011-10-20  2719.26  2720.73  2685.34  2694.01
 159 2011-10-21  2702.51  2719.12  2698.61  2712.41
 160 2011-10-24  2745.69  2776.94  2745.50  2760.95
 161 2011-10-25  2765.18  2780.24  2752.56  2769.94
 162 2011-10-27  2776.76  2862.01  2764.82  2847.57
 
 [163 rows x 5 columns],
 'india':            date         open         high          low        close
 0    2007-09-18  6136.330078  6199.839844  6117.209961  6192.879883
 1    2007-09-19  6274.509766  6403.439941  6274.509766  6399.479980
 2    2007-09-20  6408.549805  6463

In [ ]:
# Remove countries that do not have data for the specified date range (02.01.2002 to 17.09.2007) from the dictionary
addtional_columns = ['singapore', 'vietnam', 'brazil', 'malaysia']

for key in list(dfs_window_2.keys()):
  if dfs_window_2[key].empty or key in addtional_columns:
    del dfs_window_2[key]

In [ ]:
len(dfs_window_2)

9

In [ ]:
# Map dfs_window_2 to a new date range
dfs_window_2 = map_date_range(dfs_window_2, window_2_start_date, window_2_end_date)

In [ ]:
# Check for missing values in the dfs_window_2
check_missing_values(dfs_window_2)

Missing values for india:
date      0
open     60
high     60
low      60
close    60
dtype: int64


Missing values for uk:
date      0
open     35
high     35
low      35
close    35
dtype: int64


Missing values for mexico:
date      0
open     38
high     38
low      38
close    38
dtype: int64


Missing values for japan:
date      0
open     70
high     70
low      70
close    70
dtype: int64


Missing values for korea:
date      0
open     41
high     41
low      41
close    41
dtype: int64


Missing values for thailand:
date      0
open     66
high     66
low      66
close    66
dtype: int64


Missing values for switzerland:
date      0
open     31
high     31
low      31
close    31
dtype: int64


Missing values for china:
date      0
open     74
high     74
low      74
close    74
dtype: int64


Missing values for us:
date      0
open     35
high     35
low      35
close    35
dtype: int64




In [ ]:
# Perform forward filling to fill in the missing values in dfs_window_2
dfs_window_2 = forward_fill(dfs_window_2)

/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/functions/preprocessing.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


In [ ]:
# Check for missing values in the dfs_window_2
check_missing_values(dfs_window_2)

Missing values for india:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for switzerland:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for china:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for us:
date     0
open     0
high     0
low      0
close    0
dtype: int64




In [ ]:
# Calculate the volatility for each country
dfs_window_2 = calculate_volatility(dfs_window_2)

In [ ]:
# Create volatility DataFrame for window 2
df_window_2 = create_volatility_df(dfs_window_2)

In [ ]:
df_window_2.head()

,date,india,uk,mexico,japan,korea,thailand,switzerland,china,us
0,2007-09-18,15.347930,24.947423,30.099507,14.153630,17.289618,9.509994,15.241784,31.432927,21.441875
1,2007-09-19,16.146018,37.897742,26.285787,16.374742,12.477392,8.778127,20.320074,25.147074,15.914060
2,2007-09-20,15.781125,13.017135,20.668125,11.817046,14.476412,7.457364,10.597139,13.101241,8.777752
3,2007-09-21,14.732500,14.773564,16.433850,9.087991,12.643515,12.373155,13.266414,40.229595,9.769744
4,2007-09-24,17.096664,12.260918,15.371639,9.087991,12.643515,18.984209,10.884644,35.452696,11.318560


Performing ADF test

In [ ]:
for country in df_window_2.drop(columns=['date']).columns:
    adf_test(df_window_2, country)

Results of Augmented Dickey-Fuller Test for india:
ADF Statistic: -3.940031199249253
p-value: 0.0017592148247725075
Critical Values:
	1%: -3.436528314312484
	5%: -2.86426792284943
	10%: -2.568222448164332
Reject the null hypothesis. The time series india is stationary.


Results of Augmented Dickey-Fuller Test for uk:
ADF Statistic: -3.5754409467307364
p-value: 0.006248279273301726
Critical Values:
	1%: -3.436557639266102
	5%: -2.8642808573632874
	10%: -2.5682293371570823
Reject the null hypothesis. The time series uk is stationary.


Results of Augmented Dickey-Fuller Test for mexico:
ADF Statistic: -3.3363397097995287
p-value: 0.013330201314999937
Critical Values:
	1%: -3.4365341571171166
	5%: -2.8642704999764907
	10%: -2.568223820754717
Reject the null hypothesis. The time series mexico is stationary.


Results of Augmented Dickey-Fuller Test for japan:
ADF Statistic: -3.536056354728473
p-value: 0.007107961668658356
Critical Values:
	1%: -3.4365931987759417
	5%: -2.864296541617536
	

Since all the variables are stationary, no transformation is needed.

##### 28.10.2011 - 31.12.2018: Recovery from Global Financial Crisis

In [ ]:
# Check whether each DataFrame contains data from 28.10.2011 to 31.12.2018
window_3_start_date = '2011-10-28'
window_3_end_date = '2018-12-31'
check_date_range(dataframes, window_3_start_date, window_3_end_date)

{'philippines': True,
 'singapore': True,
 'india': True,
 'uk': True,
 'mexico': True,
 'japan': True,
 'vietnam': True,
 'korea': True,
 'thailand': True,
 'brazil': True,
 'malaysia': True,
 'switzerland': True,
 'china': True,
 'russia': True,
 'us': True}

Most DataFrame appears to have data from 28.10.2011 to 31.12.2018, except from Russia, which only has data from 2013. Russia will be removed from this time window.

In [ ]:
dfs_window_3 = extract_date_range(dataframes, window_3_start_date, window_3_end_date)

# Check whether each DataFrame contains data from 28.10.2011 - 31.12.2018
dfs_window_3

{'philippines':            date     open     high      low    close
 0    2011-10-28  4335.08  4345.75  4326.25  4333.72
 1    2011-11-02  4332.93  4332.93  4254.44  4260.41
 2    2011-11-03  4264.20  4286.26  4207.44  4210.25
 3    2011-11-04  4229.72  4275.70  4229.72  4271.72
 4    2011-11-08  4277.70  4322.26  4277.70  4314.67
 ...         ...      ...      ...      ...      ...
 1738 2018-12-20  7514.07  7584.44  7498.19  7563.41
 1739 2018-12-21  7536.87  7548.13  7452.24  7479.71
 1740 2018-12-26  7436.73  7453.22  7378.88  7450.01
 1741 2018-12-27  7465.04  7514.33  7447.22  7482.66
 1742 2018-12-28  7497.16  7507.29  7466.02  7466.02
 
 [1743 rows x 5 columns],
 'singapore':            date     open     high      low    close
 163  2011-10-28  2879.73  2905.72  2865.86  2905.72
 164  2011-10-31  2903.97  2903.97  2855.77  2855.77
 165  2011-11-01  2832.37  2843.58  2785.23  2789.35
 166  2011-11-02  2760.73  2836.75  2754.69  2834.75
 167  2011-11-03  2801.57  2813.48  2781.27

In [ ]:
del dfs_window_3['russia']

In [ ]:
len(dfs_window_3)

14

In [ ]:
# Map dfs_window_3 to a new date range
dfs_window_3 = map_date_range(dfs_window_3, window_3_start_date, window_3_end_date)

In [ ]:
# Check for missing values in the dfs_window_3
check_missing_values(dfs_window_3)

Missing values for philippines:
date       0
open     129
high     129
low      129
close    129
dtype: int64


Missing values for singapore:
date      0
open     60
high     60
low      60
close    60
dtype: int64


Missing values for india:
date       0
open     106
high     106
low      106
close    106
dtype: int64


Missing values for uk:
date      0
open     59
high     59
low      59
close    59
dtype: int64


Missing values for mexico:
date      0
open     76
high     76
low      76
close    76
dtype: int64


Missing values for japan:
date      0
open     89
high     89
low      89
close    89
dtype: int64


Missing values for vietnam:
date      0
open     81
high     81
low      81
close    81
dtype: int64


Missing values for korea:
date       0
open     107
high     107
low      107
close    107
dtype: int64


Missing values for thailand:
date       0
open     118
high     118
low      118
close    118
dtype: int64


Missing values for brazil:
date      0
open     93
high   

In [ ]:
# Peform forward fill to handle the missing values
dfs_window_3 = forward_fill(dfs_window_3)

/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/functions/preprocessing.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


In [ ]:
# Check for missing values in the dfs_window_3 to make sure there is no missing values left
check_missing_values(dfs_window_3)

Missing values for philippines:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for singapore:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for india:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for vietnam:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for brazil:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for malaysi

In [ ]:
# Calculate the volatility for each country
dfs_window_3 = calculate_volatility(dfs_window_3)

In [ ]:
# Create volatility DataFrame for window 3
df_window_3 = create_volatility_df(dfs_window_3)

In [ ]:
df_window_3.head()

,date,philippines,singapore,india,uk,mexico,japan,vietnam,korea,thailand,brazil,malaysia,switzerland,china,us
0,2011-10-28,6.315016,16.199842,12.223430,15.051259,30.937386,12.915482,0.0,24.251033,21.275492,1.119956,9.456700,8.458869,19.643091,11.055033
1,2011-10-31,6.315016,12.347315,9.431441,20.732088,41.301750,25.412885,0.0,27.547061,17.125333,9.373742,15.625040,17.317385,14.221694,18.701896
2,2011-11-01,6.315016,22.684887,17.590073,46.070291,36.492140,17.069397,0.0,22.147947,29.342548,16.361129,8.238955,30.762347,27.619681,25.294086
3,2011-11-02,15.894086,26.470606,15.431678,24.910615,36.492140,6.756631,0.0,23.873419,23.640117,16.361129,14.816243,19.461350,27.942278,19.110816
4,2011-11-03,21.203987,15.825074,15.838056,39.378283,23.747541,6.756631,0.0,17.379877,16.728207,1.509353,18.469910,20.583322,19.011037,23.265613


Performing ADF test

In [ ]:
for country in df_window_3.drop(columns=['date']).columns:
    adf_test(df_window_3, country)

Results of Augmented Dickey-Fuller Test for philippines:
ADF Statistic: -7.017777353801121
p-value: 6.663553313881037e-10
Critical Values:
	1%: -3.4338725134861083
	5%: -2.863095992014326
	10%: -2.5675983545064196
Reject the null hypothesis. The time series philippines is stationary.


Results of Augmented Dickey-Fuller Test for singapore:
ADF Statistic: -5.5470459696270416
p-value: 1.6518366846151787e-06
Critical Values:
	1%: -3.433876312517571
	5%: -2.863097669161313
	10%: -2.567599247525482
Reject the null hypothesis. The time series singapore is stationary.


Results of Augmented Dickey-Fuller Test for india:
ADF Statistic: -7.543663929037819
p-value: 3.326914566330679e-11
Critical Values:
	1%: -3.4338687226315336
	5%: -2.863094318475046
	10%: -2.5675974634086765
Reject the null hypothesis. The time series india is stationary.


Results of Augmented Dickey-Fuller Test for uk:
ADF Statistic: -6.3817642843511875
p-value: 2.2133954098005636e-08
Critical Values:
	1%: -3.433876312517571

All of the variables are stationary. Therefore, no transformation is needed.

##### 02.01.2019 - 31.12.2022: COVID-19 pandemic

In [ ]:
# Check whether each DataFrame contains data from 02.01.2019 to 31.12.2022
window_4_start_date = '2019-01-02'
window_4_end_date = '2022-12-31'
check_date_range(dataframes, window_4_start_date, window_4_end_date)

{'philippines': True,
 'singapore': True,
 'india': True,
 'uk': True,
 'mexico': True,
 'japan': True,
 'vietnam': True,
 'korea': True,
 'thailand': True,
 'brazil': True,
 'malaysia': True,
 'switzerland': True,
 'china': True,
 'russia': True,
 'us': True}

In [ ]:
dfs_window_4 = extract_date_range(dataframes, window_4_start_date, window_4_end_date)

# Check whether each DataFrame contains data from 02.01.2019 to 31.12.2022
dfs_window_4

{'philippines':            date     open     high      low    close
 1743 2019-01-02  7496.57  7540.26  7465.76  7489.20
 1744 2019-01-03  7507.48  7680.60  7490.28  7680.60
 1745 2019-01-04  7657.18  7801.50  7657.18  7761.11
 1746 2019-01-07  7800.88  7900.70  7787.66  7787.66
 1747 2019-01-08  7820.26  7826.08  7702.12  7702.12
 ...         ...      ...      ...      ...      ...
 2724 2022-12-23  6567.92  6567.92  6536.67  6541.03
 2725 2022-12-26  6541.03  6541.03  6541.03  6541.03
 2726 2022-12-27  6549.09  6579.26  6519.90  6564.90
 2727 2022-12-28  6567.13  6576.71  6530.27  6566.54
 2728 2022-12-29  6553.67  6591.28  6529.69  6566.39
 
 [986 rows x 5 columns],
 'singapore':            date     open     high      low    close
 1975 2019-01-02  3072.99  3081.01  3028.84  3038.89
 1976 2019-01-03  3022.27  3041.03  3002.40  3012.88
 1977 2019-01-04  2994.02  3059.23  2993.42  3059.23
 1978 2019-01-07  3101.26  3104.32  3084.88  3102.80
 1979 2019-01-08  3108.38  3126.64  3099.33 

In [ ]:
# Map dfs_window_4 to a new date range
dfs_window_4 = map_date_range(dfs_window_4, window_4_start_date, window_4_end_date)

In [ ]:
# Check for missing values in the dfs_window_4
check_missing_values(dfs_window_4)

Missing values for philippines:
date      0
open     57
high     57
low      57
close    57
dtype: int64


Missing values for singapore:
date       0
open     167
high     167
low      167
close    167
dtype: int64


Missing values for india:
date      0
open     56
high     56
low      56
close    56
dtype: int64


Missing values for uk:
date      0
open     75
high     75
low      75
close    75
dtype: int64


Missing values for mexico:
date      0
open     35
high     35
low      35
close    35
dtype: int64


Missing values for japan:
date      0
open     71
high     71
low      71
close    71
dtype: int64


Missing values for vietnam:
date      0
open     42
high     42
low      42
close    42
dtype: int64


Missing values for korea:
date      0
open     57
high     57
low      57
close    57
dtype: int64


Missing values for thailand:
date      0
open     74
high     74
low      74
close    74
dtype: int64


Missing values for brazil:
date      0
open     50
high     50
low      5

In [ ]:
# Peforming forward filling imputation to handle the missing values
dfs_window_4 = forward_fill(dfs_window_4)

/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/functions/preprocessing.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


In [ ]:
# Check for missing values in the dfs_window_4 to make sure that there is no missing data left
check_missing_values(dfs_window_4)

Missing values for philippines:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for singapore:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for india:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     2
high     2
low      2
close    2
dtype: int64


Missing values for vietnam:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for brazil:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for malaysi

In [ ]:
# Perform backward filling imputation to handle the remaining missing values
dfs_window_4 = backward_fill(dfs_window_4)

/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/functions/preprocessing.py:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


In [ ]:
# Check for missing values in the dfs_window_4 to make sure that there is no missing data left
check_missing_values(dfs_window_4)

Missing values for philippines:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for singapore:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for india:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for vietnam:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for brazil:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for malaysi

In [ ]:
# Calculate the volatility for each country
dfs_window_4 = calculate_volatility(dfs_window_4)

In [ ]:
# Create volatility DataFrame for window 4
df_window_4 = create_volatility_df(dfs_window_4)

Perform ADF test

In [ ]:
for country in df_window_4.drop(columns=['date']).columns:
    adf_test(df_window_4, country)

Results of Augmented Dickey-Fuller Test for philippines:
ADF Statistic: -6.882410499772994
p-value: 1.422328534237237e-09
Critical Values:
	1%: -3.4366839058058387
	5%: -2.8643365489845647
	10%: -2.5682589990431515
Reject the null hypothesis. The time series philippines is stationary.


Results of Augmented Dickey-Fuller Test for singapore:
ADF Statistic: -4.281213566385517
p-value: 0.00047869051731638264
Critical Values:
	1%: -3.436714730058834
	5%: -2.8643501440982058
	10%: -2.5682662399849185
Reject the null hypothesis. The time series singapore is stationary.


Results of Augmented Dickey-Fuller Test for india:
ADF Statistic: -5.289783098456346
p-value: 5.748888919705902e-06
Critical Values:
	1%: -3.436677776748241
	5%: -2.8643338457318848
	10%: -2.5682575592567196
Reject the null hypothesis. The time series india is stationary.


Results of Augmented Dickey-Fuller Test for uk:
ADF Statistic: -4.43845416734798
p-value: 0.0002536181733548275
Critical Values:
	1%: -3.4367646777960137

All of the variables are stationary. Therefore, no transformation is needed.

##### 02.01.2023 - 30.04.2024: Recovery from COVID-19 pandemic

In [ ]:
# Check whether each DataFrame contains data from 02.01.2023 to 30.04.2024
window_5_start_date = '2023-01-02'
window_5_end_date = '2024-04-30'
check_date_range(dataframes, window_5_start_date, window_5_end_date)

{'philippines': True,
 'singapore': True,
 'india': True,
 'uk': True,
 'mexico': True,
 'japan': True,
 'vietnam': True,
 'korea': True,
 'thailand': True,
 'brazil': True,
 'malaysia': True,
 'switzerland': True,
 'china': True,
 'russia': True,
 'us': True}

In [ ]:
dfs_window_5 = extract_date_range(dataframes, window_5_start_date, window_5_end_date)

# Check whether each DataFrame contains data from 02.01.2023 to 30.04.2024
dfs_window_5

{'philippines':            date     open     high      low    close
 2729 2023-01-03  6555.40  6624.76  6550.17  6586.01
 2730 2023-01-04  6601.50  6755.27  6601.50  6718.50
 2731 2023-01-05  6738.93  6767.05  6730.52  6761.33
 2732 2023-01-06  6736.65  6736.65  6666.68  6667.97
 2733 2023-01-09  6692.30  6790.24  6685.07  6790.24
 ...         ...      ...      ...      ...      ...
 3050 2024-04-24  6538.98  6593.18  6538.98  6572.75
 3051 2024-04-25  6578.20  6589.89  6574.88  6574.88
 3052 2024-04-26  6557.95  6628.75  6547.76  6628.75
 3053 2024-04-29  6629.57  6769.64  6629.57  6769.64
 3054 2024-04-30  6749.82  6776.54  6700.49  6700.49
 
 [326 rows x 5 columns],
 'singapore':            date     open     high      low    close
 2851 2023-01-03  3243.99  3252.12  3212.63  3245.80
 2852 2023-01-04  3251.42  3256.37  3238.70  3242.46
 2853 2023-01-05  3264.53  3297.34  3262.38  3292.66
 2854 2023-01-06  3290.32  3290.32  3271.08  3276.72
 2855 2023-01-09  3311.67  3343.47  3291.04 

In [ ]:
# Map dfs_window_4 to a new date range
dfs_window_5 = map_date_range(dfs_window_5, window_5_start_date, window_5_end_date)

In [ ]:
# Check for missing values in the dfs_window_5
check_missing_values(dfs_window_5)

Missing values for philippines:
date      0
open     21
high     21
low      21
close    21
dtype: int64


Missing values for singapore:
date      0
open     17
high     17
low      17
close    17
dtype: int64


Missing values for india:
date      0
open     22
high     22
low      22
close    22
dtype: int64


Missing values for uk:
date      0
open     12
high     12
low      12
close    12
dtype: int64


Missing values for mexico:
date      0
open     14
high     14
low      14
close    14
dtype: int64


Missing values for japan:
date      0
open     22
high     22
low      22
close    22
dtype: int64


Missing values for vietnam:
date      0
open     20
high     20
low      20
close    20
dtype: int64


Missing values for korea:
date      0
open     20
high     20
low      20
close    20
dtype: int64


Missing values for thailand:
date      0
open     24
high     24
low      24
close    24
dtype: int64


Missing values for brazil:
date      0
open     16
high     16
low      16
clo

In [ ]:
# Perform forward filling imputation
dfs_window_5 = forward_fill(dfs_window_5)

/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/functions/preprocessing.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


In [ ]:
# Check for missing values in the dfs_window_5 again to make sure there is no missing values left
check_missing_values(dfs_window_5)

Missing values for philippines:
date     0
open     1
high     1
low      1
close    1
dtype: int64


Missing values for singapore:
date     0
open     1
high     1
low      1
close    1
dtype: int64


Missing values for india:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for uk:
date     0
open     1
high     1
low      1
close    1
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     2
high     2
low      2
close    2
dtype: int64


Missing values for vietnam:
date     0
open     1
high     1
low      1
close    1
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     1
high     1
low      1
close    1
dtype: int64


Missing values for brazil:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for malaysi

In [ ]:
# Perform backward filling to handle the remaining missing values
dfs_window_5 = backward_fill(dfs_window_5)

/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/functions/preprocessing.py:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


In [ ]:
# Check for missing values in the dfs_window_5 again to make sure there is no missing values left
check_missing_values(dfs_window_5)

Missing values for philippines:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for singapore:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for india:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for vietnam:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for brazil:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for malaysi

In [ ]:
# Calculate the volatility for each country
dfs_window_5 = calculate_volatility(dfs_window_5)

In [ ]:
# Create volatility DataFrame for window 5
df_window_5 = create_volatility_df(dfs_window_5)

Peform ADF test

In [ ]:
for country in df_window_5.drop(columns=['date']).columns:
    adf_test(df_window_5, country)

Results of Augmented Dickey-Fuller Test for philippines:
ADF Statistic: -10.036505874101831
p-value: 1.5356548408736335e-17
Critical Values:
	1%: -3.4494474563375737
	5%: -2.8699542285903887
	10%: -2.5712527305187987
Reject the null hypothesis. The time series philippines is stationary.


Results of Augmented Dickey-Fuller Test for singapore:
ADF Statistic: -5.055641762816369
p-value: 1.7202968182098525e-05
Critical Values:
	1%: -3.449962981927952
	5%: -2.870180642420163
	10%: -2.5713734527352607
Reject the null hypothesis. The time series singapore is stationary.


Results of Augmented Dickey-Fuller Test for india:
ADF Statistic: -7.425478394492573
p-value: 6.566064316202762e-11
Critical Values:
	1%: -3.4495033946549123
	5%: -2.8699787979596136
	10%: -2.5712658305164955
Reject the null hypothesis. The time series india is stationary.


Results of Augmented Dickey-Fuller Test for uk:
ADF Statistic: -6.3219785501826085
p-value: 3.050378900681858e-08
Critical Values:
	1%: -3.449559661646

All of the variables are stationary. Therefore, no transformation is needed.

## VAR Modelling

In [ ]:
# Create a list of windowed volatility DataFrames
windowed_vol_dfs = [df_window_1, df_window_2, df_window_3, df_window_4, df_window_5]

In [ ]:
for i, df_volatility in enumerate(windowed_vol_dfs):
    window = f"window_{i+1}"
    print(f"Window: {window}")
    print(df_volatility.drop(columns=['date']).corr(method='pearson'))
    print()

Window: window_1
                   uk    mexico     japan     korea  thailand  switzerland  \
uk           1.000000  0.256136  0.391186  0.369396  0.042396     0.223647   
mexico       0.256136  1.000000  0.167427  0.128424  0.060219     0.081865   
japan        0.391186  0.167427  1.000000  0.415146  0.163873     0.011168   
korea        0.369396  0.128424  0.415146  1.000000  0.136289     0.027165   
thailand     0.042396  0.060219  0.163873  0.136289  1.000000     0.006970   
switzerland  0.223647  0.081865  0.011168  0.027165  0.006970     1.000000   
us           0.688052  0.333897  0.367789  0.365930  0.076302     0.088225   

                   us  
uk           0.688052  
mexico       0.333897  
japan        0.367789  
korea        0.365930  
thailand     0.076302  
switzerland  0.088225  
us           1.000000  

Window: window_2
                india        uk    mexico     japan     korea  thailand  \
india        1.000000  0.534643  0.524582  0.487288  0.494965  0.485320  

##### Average Spillover Table

In [ ]:
for i, df_volatility in enumerate(windowed_vol_dfs):
  window = f"window_{i+1}"
  print(f"Calculating spillover for {window}...")
  # Calculate the average spillover for each window
  spillovers_table, lag_order, forecast_horizon = calculate_avg_spillover_table(
      df_volatility.drop(columns=["date"])
  )
  print(f"Finished calculating spillover for {window}!")
  print(f"Lag order: {lag_order}")
  print(f"Forecast horizon: {forecast_horizon}")
  print(f"Saving spillover table for {window}...")
  # Save the spillover table to a CSV file
  spillovers_table.to_csv(
      f"output/var/{window}_spillover_table.csv", index=True
  )
  print(f"Finished saving spillover table for {window}!")
  print()

Calculating spillover for window_1...


/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Finished calculating spillover for window_1!
Lag order: 9
Forecast horizon: 10
Saving spillover table for window_1...
Finished saving spillover table for window_1!

Calculating spillover for window_2...
Finished calculating spillover for window_2!
Lag order: 6
Forecast horizon: 10
Saving spillover table for window_2...
Finished saving spillover table for window_2!

Calculating spillover for window_3...
Finished calculating spillover for window_3!
Lag order: 4
Forecast horizon: 10
Saving spillover table for window_3...
Finished saving spillover table for window_3!

Calculating spillover for window_4...
Finished calculating spillover for window_4!
Lag order: 4
Forecast horizon: 10
Saving spillover table for window_4...
Finished saving spillover table for window_4!

Calculating spillover for window_5...
Finished calculating spillover for window_5!
Lag order: 1
Forecast horizon: 10
Saving spillover table for window_5...
Finished saving spillover table for window_5!



##### Net Pair-wise Spillover Table

In [ ]:
for i in range(5):
  window = f"window_{i+1}"
  print(f"Calculating net pair-wise spillover for {window}...")
  # Read the spillover table for the current window
  spillover_table = pd.read_csv(f"output/var/{window}_spillover_table.csv", index_col=0)
  # Calculate the net pair-wise spillover for the current window
  normalized_spillover_table = spillover_table.drop(index=['Directional TO others', 'Total Spillover Index']).drop(columns=['Directional FROM others'])
  net_pairwise_spillover_table = calculate_net_pairwise_spillover_table(normalized_spillover_table)
  print(f"Finished calculating net pair-wise spillover for {window}!")

  print(net_pairwise_spillover_table)
  print()
  # print(f"Saving net pair-wise spillover table for {window}...")
  # # Save the net pair-wise spillover table to a CSV file
  # net_pairwise_spillover_table.to_csv(f"output/var/{window}_net_pairwise_spillover_table.csv", index=True)
  # print(f"Finished saving net pair-wise spillover table for {window}!")

  

Calculating net pair-wise spillover for window_1...
Finished calculating net pair-wise spillover for window_1!
                     uk      mexico      japan      korea   thailand  \
uk             0.000000  -27.333059 -29.937095 -26.595816   2.833851   
mexico        27.333059    0.000000 -19.295053 -16.342519 -12.393385   
japan         29.937095   19.295053   0.000000 -82.254837  -7.202867   
korea         26.595816   16.342519  82.254837   0.000000  -9.207124   
thailand      -2.833851   12.393385   7.202867   9.207124   0.000000   
switzerland  267.905726   -5.122709  -1.346795  -3.226773   2.326723   
us            78.753251  107.505077 -14.094306  -9.393587  -2.214721   

             switzerland          us  
uk           -267.905726  -78.753251  
mexico          5.122709 -107.505077  
japan           1.346795   14.094306  
korea           3.226773    9.393587  
thailand       -2.326723    2.214721  
switzerland     0.000000   -8.934079  
us              8.934079    0.000000  
